## 4.9 Task - Part 1

##### This script contains the following points:

#### 3. Import libraries and data

#### 4. Wrangle data

#### 5. Complete fundamental data quality and consistency checks

#### 6. Combine customer data with prepared IC data

#### 7. Export data as pickle

# ____________________________________________________________________

####

### 3. Import libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Set path
path = r'/Users/mainframe/Documents/Instacart Basket Analysis'

In [3]:
# Import Data
df_customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
# Check data
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_customer.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [6]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


### 4. Wrangle data

In [7]:
# Drop 'First Name' and 'Surnam' columns
df_customer = df_customer.drop(columns = ['First Name', 'Surnam'])

In [8]:
# Re-name columns
df_customer.rename(columns = {'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'dependents', 'fam_status' : 'marital'}, inplace = True)

In [9]:
# Check columns
print(df_customer.columns)

Index(['user_id', 'gender', 'state', 'age', 'date_joined', 'dependents',
       'marital', 'income'],
      dtype='object')


### 5. Complete fundamental data quality and consistency checks

In [10]:
# Check for mixed-type data
for col in df_customer.columns.tolist():
    weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customer[weird]) > 0:
        print (col)

/var/folders/rg/l7v73y996hv36vxgfkdf06lc0000gp/T/ipykernel_72776/2126102000.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
/var/folders/rg/l7v73y996hv36vxgfkdf06lc0000gp/T/ipykernel_72776/2126102000.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
/var/folders/rg/l7v73y996hv36vxgfkdf06lc0000gp/T/ipykernel_72776/2126102000.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
/var/folders/rg/l7v73y996hv36vxgfkdf06lc0000gp/T/ipykernel_72776/2126102000.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_customer[[col]].applymap

In [11]:
# Check for missing values
df_customer.isnull().sum()

user_id        0
gender         0
state          0
age            0
date_joined    0
dependents     0
marital        0
income         0
dtype: int64

In [12]:
# Check for duplicates
df_dups = df_customer[df_customer.duplicated()]

In [13]:
df_dups

,user_id,gender,state,age,date_joined,dependents,marital,income


### 6. Combine customer data with rest of prepared IC data

In [14]:
# Change data types
df_customer['user_id'] = df_customer['user_id'].astype('uint32')
df_customer['age'] = df_customer['age'].astype('uint8')
df_customer['dependents'] = df_customer['dependents'].astype('uint8')
df_customer['income'] = df_customer['income'].astype('uint32')

In [15]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      206209 non-null  uint32
 1   gender       206209 non-null  object
 2   state        206209 non-null  object
 3   age          206209 non-null  uint8 
 4   date_joined  206209 non-null  object
 5   dependents   206209 non-null  uint8 
 6   marital      206209 non-null  object
 7   income       206209 non-null  uint32
dtypes: object(4), uint32(2), uint8(2)
memory usage: 8.3+ MB


In [16]:
# Import prepared IC data
prepped_IC_df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'opmd_outliers_dropped.pkl'))

In [17]:
# Check data
prepped_IC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   order_id                       uint32  
 1   user_id                        uint32  
 2   order_number                   uint8   
 3   orders_day_of_week             uint8   
 4   hour_ordered                   uint8   
 5   days_since_prior_order         float16 
 6   product_id                     uint16  
 7   add_to_cart_order              uint8   
 8   reordered                      int8    
 9   product_name                   object  
 10  aisle_id                       uint8   
 11  department_id                  uint8   
 12  prices                         float64 
 13  _merge                         category
 14  price_range_loc                object  
 15  busiest_day                    object  
 16  busiest_days                   object  
 17  busiest_period_of_day    

In [18]:
# Drop _merge column
prepped_IC_df = prepped_IC_df.drop(columns = ['_merge'])

In [19]:
# Merge the dataframes 'prepped_IC_df' and 'df_customer'
opc_IC_df = prepped_IC_df.merge(df_customer, on = 'user_id', indicator = True)

In [20]:
opc_IC_df.head()

,order_id,user_id,order_number,orders_day_of_week,hour_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,median_days_since_prior_order,frequency_flag,gender,state,age,date_joined,dependents,marital,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [21]:
# Confirm results with merge flag
opc_IC_df['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

### 8. Export data frame as pickle file

In [22]:
# Export as .pkl
opc_IC_df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'IC_data.pkl'))